In [1]:
import numpy as np
import pandas as pd
import glob
import scipy.misc
import tensorflow as tf
import scipy
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

## Explore Bored Apes data

In [4]:
df_apes = pd.read_csv("bored_apes.csv")
df_apes.drop('Unnamed: 0', axis=1, inplace=True)
df_apes

,image_id,trait_type,value
0,0,Earring,Silver Hoop
1,0,Background,Orange
2,0,Fur,Robot
3,0,Clothes,Striped Tee
4,0,Mouth,Discomfort
...,...,...,...
58808,9999,Background,Purple
58809,9999,Mouth,Bored Unshaven
58810,9999,Earring,Gold Hoop
58811,9999,Fur,Gray


In [6]:
print(df_apes.trait_type.unique(), len(df_apes.trait_type.unique()))

['Earring' 'Background' 'Fur' 'Clothes' 'Mouth' 'Eyes' 'Hat'] 7


In [7]:
print(df_apes.value.unique(), len(df_apes.value.unique()))

['Silver Hoop' 'Orange' 'Robot' 'Striped Tee' 'Discomfort' 'X Eyes' 'Grin'
 'Vietnam Jacket' 'Blue Beams' '3d' 'Bored Cigarette' "Sea Captain's Hat"
 'Aquamarine' 'Purple' 'Bored' 'Tongue Out' 'Bone Necklace' 'Cheetah'
 'Navy Striped Tee' 'Phoneme L' 'Party Hat 2' 'Golden Brown' 'Closed'
 'Blue' 'Bayc Flipped Brim' 'Brown' 'Army Green' 'Bayc T Red'
 'Dumbfounded' 'Tweed Suit' 'Cream' 'Crazy' 'Yellow' 'S&m Hat' 'Gray'
 'Stuntman Helmet' 'Angry' 'Wool Turtleneck' 'Beanie' 'Gold Stud' 'Zombie'
 'Silver Stud' 'Sleepy' 'Small Grin' "Seaman's Hat" 'Stunt Jacket'
 'Bayc Hat Red' 'Dmt' 'Eyepatch' 'Dark Brown' 'Laurel Wreath'
 'Bored Unshaven Cigarette' 'Smoking Jacket' 'Bloodshot' 'Black Holes T'
 'Wide Eyed' 'Black' 'Bone Tee' 'Coins' 'Police Motorcycle Helmet'
 'Diamond Stud' 'Tanktop' 'Black T' "Girl's Hair Pink" 'Horns'
 'Bored Unshaven' 'New Punk Blue' 'Bayc T Black' 'Commie Hat' 'Tan'
 'Cyborg' 'Bayc Hat Black' 'Rage' 'Bandolier' 'Phoneme Vuh' 'Phoneme  ooo'
 'Tuxedo Tee' 'Bored Pipe' 'T

In [8]:
df_apes["label"] = df_apes["trait_type"] + " " + df_apes["value"]
df_apes

,image_id,trait_type,value,label
0,0,Earring,Silver Hoop,Earring Silver Hoop
1,0,Background,Orange,Background Orange
2,0,Fur,Robot,Fur Robot
3,0,Clothes,Striped Tee,Clothes Striped Tee
4,0,Mouth,Discomfort,Mouth Discomfort
...,...,...,...,...
58808,9999,Background,Purple,Background Purple
58809,9999,Mouth,Bored Unshaven,Mouth Bored Unshaven
58810,9999,Earring,Gold Hoop,Earring Gold Hoop
58811,9999,Fur,Gray,Fur Gray


In [9]:
len(df_apes.label.unique())

168

In [12]:
df_apes.drop(['trait_type', 'value'], axis=1, inplace=True)
df_apes

,image_id,label
0,0,Earring Silver Hoop
1,0,Background Orange
2,0,Fur Robot
3,0,Clothes Striped Tee
4,0,Mouth Discomfort
...,...,...
58808,9999,Background Purple
58809,9999,Mouth Bored Unshaven
58810,9999,Earring Gold Hoop
58811,9999,Fur Gray


In [14]:
df_apes.to_csv("apes_modified.csv", index=False)

## Punks

In [15]:
df_punks = pd.read_csv("cryptopunks.csv")
df_punks.drop('Unnamed: 0', axis=1, inplace=True)
df_punks

,image_id,trait_type,value
0,0,type,Female
1,0,accessory,Green Eye Shadow
2,0,accessory,Earring
3,0,accessory,Blonde Bob
4,0,accessory,3 attributes
...,...,...,...
33520,9998,accessory,Clown Eyes Green
33521,9998,type,Female
33522,9998,accessory,Black Lipstick
33523,9998,accessory,Wild White Hair


In [16]:
print(df_punks.trait_type.unique(), len(df_punks.trait_type.unique()))

['type' 'accessory'] 2


In [17]:
print(df_punks.value.unique(), len(df_punks.value.unique()))

['Female' 'Green Eye Shadow' 'Earring' 'Blonde Bob' '3 attributes'
 'Mohawk' 'Blue Eye Shadow' '2 attributes' 'Tassle Hat' '1 attributes'
 'Male' 'Peak Spike' 'Nerd Glasses' 'Luxurious Beard' 'Mohawk Dark'
 'Frumpy Hair' 'Messy Hair' 'Black Lipstick' 'Regular Shades'
 'Purple Hair' 'Wild Hair' 'Vampire Hair' 'Cap Forward' 'Cap' 'Police Cap'
 'Stringy Hair' 'Eye Patch' 'Mohawk Thin' 'Clown Eyes Green' 'Fedora'
 'Silver Chain' 'Mole' 'Goat' 'Top Hat' 'Front Beard' 'Chinstrap'
 'Headband' 'Ape' 'Hot Lipstick' 'Clown Eyes Blue' 'Bandana' 'Big Shades'
 'Horned Rim Glasses' 'Mustache' 'Muttonchops' 'Small Shades'
 'Straight Hair Dark' 'Hoodie' 'Normal Beard Black' 'Classic Shades'
 'Orange Side' 'Crazy Hair' '4 attributes' 'Vape' 'Purple Eye Shadow'
 'Half Shaved' 'Purple Lipstick' 'VR' 'Normal Beard' 'Wild Blonde' 'Smile'
 'Big Beard' 'Handlebars' 'Front Beard Dark' 'Straight Hair Blonde'
 'Shadow Beard' 'Cigarette' 'Pink With Hat' 'Eye Mask' 'Cowboy Hat'
 'Shaved Head' 'Blonde Short' 'Clow

In [18]:
df_punks["label"] = df_punks["trait_type"] + " " + df_punks["value"]
df_punks

,image_id,trait_type,value,label
0,0,type,Female,type Female
1,0,accessory,Green Eye Shadow,accessory Green Eye Shadow
2,0,accessory,Earring,accessory Earring
3,0,accessory,Blonde Bob,accessory Blonde Bob
4,0,accessory,3 attributes,accessory 3 attributes
...,...,...,...,...
33520,9998,accessory,Clown Eyes Green,accessory Clown Eyes Green
33521,9998,type,Female,type Female
33522,9998,accessory,Black Lipstick,accessory Black Lipstick
33523,9998,accessory,Wild White Hair,accessory Wild White Hair


In [19]:
len(df_punks.label.unique())

100

In [20]:
df_punks.drop(['trait_type', 'value'], axis=1, inplace=True)
df_punks

,image_id,label
0,0,type Female
1,0,accessory Green Eye Shadow
2,0,accessory Earring
3,0,accessory Blonde Bob
4,0,accessory 3 attributes
...,...,...
33520,9998,accessory Clown Eyes Green
33521,9998,type Female
33522,9998,accessory Black Lipstick
33523,9998,accessory Wild White Hair


In [22]:
df_punks.to_csv("punks_modified.csv", index=False)